# Decision Tree Regressor 

## Mục tiêu:
- Xây dựng mô hình Decision Tree Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 


## Giới thiệu

Mô hình cây quyết định phân chia không gian đặc trưng thành nhiều vùng và dự đoán bằng giá trị trung bình của từng vùng.

### **Ưu điểm:**

- Dễ hiểu, trực quan.

- Không cần chuẩn hóa dữ liệu.

- Học được quan hệ phi tuyến.

### **Nhược điểm:**

- Rất dễ overfitting.

- Không ổn định: dễ bị ảnh hưởng bởi nhiễu/outliers.

## Bước 1 - Import các thư viện cần thiết

In [8]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [9]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 15)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Romania,ROU,2019,-0.100387,-0.593216,-1.101137,75.607317,-0.107887,0.079336,1.142611,0.621932,0.746494,-0.047183,-0.857182,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.685495,-0.500838,72.432195,-0.405891,0.241659,0.204355,0.586683,0.718065,-0.221970,1.767314,-0.189257
2,Angola,AGO,2008,-0.083899,0.283435,1.500859,55.281000,-0.470953,1.327705,-0.839543,-1.545881,-2.268636,-0.432669,0.305989,1.598947
3,Albania,ALB,2001,-0.222240,-0.619770,-1.349940,75.083000,-0.584364,0.930412,-0.835544,0.600783,-0.024271,-0.404910,0.108810,-0.104497
4,Central African Republic,CAF,2021,-0.206911,2.567341,0.236338,40.279000,-0.618434,-0.427187,-1.336901,-2.349558,-3.008624,-0.529029,1.729562,1.172890


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [10]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 12

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. poverty_ratio
  4. pop_growth
  5. gdp_per_capita
  6. gdp_growth
  7. sanitation
  8. electricity
  9. water_access
  10. co2_emissions
  11. slum_population
  12. labor_force

Kích thước X_train: (3255, 12)
Kích thước y_train: (3255,)


## Bước 4 - Xây dựng và huấn luyện mô hình Decision Tree 

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Decision Tree.

In [ ]:
from scipy.stats import randint, uniform

# Định nghĩa không gian siêu tham số để tìm kiếm
param_distributions = {
    "max_depth": randint(2, 40),
    "min_samples_split": randint(2, 50),
    "min_samples_leaf": randint(1, 20),
    "max_features": uniform(0.3, 0.7),   # 0.3 → 1.0
    "min_weight_fraction_leaf": uniform(0.0, 0.1),
    "max_leaf_nodes": randint(10, 200),
    "min_impurity_decrease": uniform(0.0, 0.02)
}

# Tạo RandomizedSearchCV
grid_search = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_distributions=param_distributions,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    n_iter=60,
    random_state=42
)

# Thực hiện tìm kiếm
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


,estimator,DecisionTreeR...ndom_state=42)
,param_distributions,"{'max_depth': <scipy.stats....001F9F71D5400>, 'max_features': <scipy.stats....001F9B7E857F0>, 'max_leaf_nodes': <scipy.stats....001F9B7E36190>, 'min_impurity_decrease': <scipy.stats....001F9B7F04CD0>, ...}"
,n_iter,60
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [5]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_dt = grid_search.best_estimator_
best_params = grid_search.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về cây tối ưu
print(f"\nĐộ sâu của cây tối ưu: {best_dt.get_depth()}")
print(f"Số lượng lá: {best_dt.get_n_leaves()}")

SIÊU THAM SỐ TỐI ƯU
  max_depth           : 21
  max_features        : 0.7064806350055182
  max_leaf_nodes      : 121
  min_impurity_decrease: 0.002029430857320642
  min_samples_leaf    : 3
  min_samples_split   : 26
  min_weight_fraction_leaf: 0.0005061583846218687

Độ sâu của cây tối ưu: 17
Số lượng lá: 121


### 4.3. Đánh giá mô hình tối ưu trên tập train

In [6]:
y_pred = best_dt.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
train_r2 = r2_score(y_train, y_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Decision Tree:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 loss: {train_r2:.3f}")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Decision Tree:
RMSE loss: 2.210
MAE loss: 1.554
R2 loss: 0.935


### 4.4. Lưu mô hình Decision Tree đã tối ưu vào file để sử dụng sau này.

In [7]:
# Tạo thư mục lưu model nếu chưa có
os.makedirs('../model/3_decision_tree', exist_ok=True)

# Lưu mô hình
model_path = '../model/3_decision_tree/decision_tree.pkl'
joblib.dump(best_dt, model_path) 

['../model/3_decision_tree/decision_tree.pkl']

## Kết luận

### Tổng kết:
- Mô hình Decision Tree đã được xây dựng và tối ưu hóa thành công
- RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
- Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2